In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from time import time
import datetime
import gc
from scipy.signal import find_peaks, peak_widths, peak_prominences
# Model
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
# Ensemble
from scipy.optimize import minimize
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
from google.colab import drive
drive.mount('/content/drvie')

Drive already mounted at /content/drvie; to attempt to forcibly remount, call drive.mount("/content/drvie", force_remount=True).


In [25]:
train = pd.read_csv('/content/drvie/My Drive/Colab Notebooks/Data/train.csv',index_col=0)
test = pd.read_csv('/content/drvie/My Drive/Colab Notebooks/Data/test.csv',index_col=0)
train.head()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0
1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1
2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0
3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0
4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0


In [26]:
# colnames = ['u', 'g', 'r', 'i', 'z', 'redshift', 'dered_u', 'dered_g', 'dered_r',
#        'dered_i', 'dered_z', 'nObserve', 'nDetect', 'airmass_u', 'airmass_g',
#        'airmass_r', 'airmass_i', 'airmass_z']
# X, y = train[colnames], train['class']
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=777)

In [27]:
# xgb_model = xgb.XGBClassifier(seed = 777,
#                            learning_rate=0.015,
#                            colsample_bytree=0.5232334448672797,
#                            gamma=0.8661895281603577,
#                            reg_alpha=3.54036580825445,
#                            reg_lambda=0.10293226563406928,
#                            subsample=0.7849549260809972,
#                            max_depth=10,
#                            tree_method='gpu_hist',
#                            n_estimators=1500)
# xgb_pred = xgb_model.fit(X_train,y_train).predict(X_test)
# xgb_accuracy = accuracy_score(y_test, xgb_pred)

# print("Acc : {}".format(xgb_accuracy))

In [28]:
target = 'class'
IDcol = 'id'

In [29]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=100):
   
    # get new n_estimator
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='error', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(alg)
    
    # Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['class'], eval_metric='error')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Training Accuracy : %.4g" % metrics.accuracy_score(dtrain['class'].values, dtrain_predictions))

In [30]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
predictors

['u',
 'g',
 'r',
 'i',
 'z',
 'redshift',
 'dered_u',
 'dered_g',
 'dered_r',
 'dered_i',
 'dered_z',
 'nObserve',
 'nDetect',
 'airmass_u',
 'airmass_g',
 'airmass_r',
 'airmass_i',
 'airmass_z']

In [31]:
param_test1 = {
 'max_depth':range(3,10,3),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=100, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  tree_method='gpu_hist',
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019),
param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   40.1s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([ 2.99958   ,  2.81989779,  2.77426829,  5.34207382,  5.10301461,
          4.97862396, 15.82031684, 14.07853079, 11.37457156]),
  'mean_score_time': array([0.04012094, 0.03872919, 0.03677139, 0.03826113, 0.04010177,
         0.03853855, 0.03999643, 0.04253731, 0.03818765]),
  'mean_test_score': array([0.90916562, 0.90909688, 0.90900625, 0.91955625, 0.91931562,
         0.91929375, 0.92526562, 0.92514687, 0.92497187]),
  'param_max_depth': masked_array(data=[3, 3, 3, 6, 6, 6, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[1, 3, 5, 1, 3, 5, 1, 3, 5],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 3, 'min_child_weight': 1},
   {'max_depth': 3, 'min_child_weig

In [33]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator =  xgb.XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=100, 
                                                  max_depth=9, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  tree_method='gpu_hist',
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019), 
                        param_grid = param_test2, scoring='accuracy', n_jobs=-1, iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([16.13878937, 15.9862535 , 15.98022556, 16.08943143, 13.682654  ]),
  'mean_score_time': array([0.04412384, 0.04607058, 0.0438067 , 0.04416361, 0.03968649]),
  'mean_test_score': array([0.92526562, 0.92526562, 0.92526562, 0.92526562, 0.92526562]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'rank_test_score': array([1, 1, 1, 1, 1], dtype=int32),
  'split0_test_score': array([0.92540625, 0.92540625, 0.92540625, 0.92540625, 0.92540625]),
  'split1_test_score': array([0.92690625, 0.92690625, 0.92690625, 0.92690625, 0.92690625]),
  'split2_test_score': array([0.92564062, 0.92564062, 0.92564062, 0.92564062, 0.92564062]),
  'split3_test_score': array([0.92451562, 0.92451562, 0.92451562, 0.92451562, 0.92451562]),
  'split4_test_score':

In [34]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=100, 
                                                  max_depth=9, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  tree_method='gpu_hist',
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019), 
                        param_grid = param_test3, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch3.fit(train[predictors],train[target])
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 10.9min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([17.57342334, 17.87636495, 18.14020395, 18.45186176, 15.68960133,
         16.14527516, 16.4784749 , 16.89016047, 15.19043384, 15.52514462,
         15.79456797, 16.15406547, 14.43656492, 14.88623714, 15.05101666,
         14.73825216]),
  'mean_score_time': array([0.04184585, 0.04377179, 0.04295688, 0.04606328, 0.04086046,
         0.04043846, 0.04153109, 0.04349365, 0.03949189, 0.04066086,
         0.04007521, 0.04070568, 0.03998895, 0.03976793, 0.04110718,
         0.03703938]),
  'mean_test_score': array([0.92470938, 0.92485   , 0.92485313, 0.92465938, 0.92491875,
         0.9251375 , 0.92514375, 0.92496875, 0.92522812, 0.92542187,
         0.92526562, 0.92528125, 0.92532813, 0.9253625 , 0.9252    ,
         0.9250625 ]),
  'param_colsample_bytree': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8,
                     0.8, 0.9, 0.9, 0.9, 0.9],
               mask=[False, False, False, False, False, False, False, False,
             

In [ ]:
param_test4 = {
 'subsample':[i/100.0 for i in range(40,80)],
}
gsearch4 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.01, 
                                                  n_estimators=100, 
                                                  max_depth=9, 
                                                  min_child_weight=1, 
                                                  gamma=0.0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.7,
                                                  tree_method='gpu_hist',
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019), 
                        param_grid = param_test4, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch4.fit(train[predictors],train[target])
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

In [37]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = xgb.XGBClassifier( 
                                                  n_estimators=500, 
                                                  max_depth=9, 
                                                  min_child_weight=1, 
                                                  gamma=0.0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.7,
                                                  tree_method='gpu_hist',
                                                  nthread=-1, 
                                                  scale_pos_weight=1, seed=2019), 
                        param_grid = param_test5, scoring='accuracy', n_jobs=-1, iid=False, cv=5, verbose=10)
gsearch5.fit(train[predictors],train[target])
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   13.7s


KeyboardInterrupt: ignored